### TS imputation Invocation Notebook - LLM based methods - example 

In [ ]:
import subprocess
from stelar.client import Client, Dataset, TaskSpec
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


c = Client(base_url="https://klms.stelar.gr", username="***", password="***")

In [ ]:
d = c.datasets["lai-timeseries-experiments"]

In [3]:
d.resources

Resource[UUID('7753db69-321e-4548-834a-1c9b4dfeba7e'), UUID('1143e073-14c9-42fe-b0d8-f4ce6b21e5c9'), UUID('7d956fcb-bb18-4e38-9f04-a11346a6c5fe'), UUID('c481a6ee-885d-4027-ac6e-31744392aa88'), UUID('ff383f8d-9b88-4726-aad3-fa18f5b03c81'), UUID('c02a0fdb-3e1e-40c0-8a8f-268feadce13e'), UUID('b3a913a4-9a7e-44df-8750-bd39825912d0'), UUID('bf3e9ef8-85ad-431f-bb8c-3a5684df4e37'), UUID('6f5f3907-8e1c-4949-8b0e-712eb221a439'), UUID('d927c6eb-d449-44f8-8f17-5480313b96b3'), UUID('5201f221-955f-4dec-9e42-6b2d42d0781e'), UUID('eb185b3e-442d-4611-b524-54005dc2f5a7'), UUID('edf678ed-68fa-4858-a39e-56f6ade77bff'), UUID('00b0483a-37ee-432d-8807-505437cdd1ae'), UUID('65bedf2a-4d2b-45a4-b4d7-2352724c2ca6'), UUID('968302f3-26e9-402a-a79b-a2e388af456a'), UUID('e635fed4-1675-4601-bb99-adb897625ac7'), UUID('2046feea-a79b-4dc7-a360-c4ba293871e0'), UUID('a2a5c096-4b37-4033-917e-dcedf187432a'), UUID('358fd7a1-bebc-478e-8c9c-cbd7ec80cc27'), UUID('cfe3e921-1247-4b1b-ac1a-ebc51d30a430'), UUID('1c31dbd3-09b3-418a-

In [11]:
d.resources[0]

<Resource 7753db69-321e-4548-834a-1c9b4dfeba7e CLEAN>

In [13]:
d.id
d.name

UUID('aca5e197-6679-41c5-a978-c87b2ebde81a')

'lai-timeseries-experiments'

In [14]:
number_of_csvs = 0 

for i in d.resources:
    print(i.name)
    print((i.name).split("_"))
    print("model:", (i.name).split("_")[-3])
    print("prompt mode:", (i.name).split("_")[-2])
    print("number of time series:", (i.name).split("_")[-5])
    print("missingness percentage:", (i.name).split("_")[-4])
    print("crop_type:", (i.name).split("_")[-8])
    print("core name:", (i.name).split("_low-missing-values-combined-lai-df.csv")[0] if i.name.endswith("_low-missing-values-combined-lai-df.csv") else (i.name).split("_df-missing.csv")[0])
    print(i.id)
    print("\n")
    number_of_csvs+=1

print('number of csv:', number_of_csvs)


AUT_SF_2022_LAI_10_high_llama3.1_agriculture-enhanced_low-missing-values-combined-lai-df.csv
['AUT', 'SF', '2022', 'LAI', '10', 'high', 'llama3.1', 'agriculture-enhanced', 'low-missing-values-combined-lai-df.csv']
model: llama3.1
prompt mode: agriculture-enhanced
number of time series: 10
missingness percentage: high
crop_type: SF
core name: AUT_SF_2022_LAI_10_high_llama3.1_agriculture-enhanced
7753db69-321e-4548-834a-1c9b4dfeba7e


AUT_SF_2022_LAI_10_high_llama3.1_agriculture-enhanced_df-missing.csv
['AUT', 'SF', '2022', 'LAI', '10', 'high', 'llama3.1', 'agriculture-enhanced', 'df-missing.csv']
model: llama3.1
prompt mode: agriculture-enhanced
number of time series: 10
missingness percentage: high
crop_type: SF
core name: AUT_SF_2022_LAI_10_high_llama3.1_agriculture-enhanced
1143e073-14c9-42fe-b0d8-f4ce6b21e5c9


AUT_KM_2022_LAI_10_high_llama3.1_agriculture-enhanced_df-missing.csv
['AUT', 'KM', '2022', 'LAI', '10', 'high', 'llama3.1', 'agriculture-enhanced', 'df-missing.csv']
model: l

#### in this example the csv that we would like to impute is the one with name: AUT_KM_2022_LAI_10_high_mistral-nemo_simple_df-missing.csv and id -> bf3e9ef8-85ad-431f-bb8c-3a5684df4e37 and the ground truth is the one with name: AUT_KM_2022_LAI_10_high_mistral-nemo_simple_low-missing-values-combined-lai-df.csv and id -> edf678ed-68fa-4858-a39e-56f6ade77bff

In [ ]:
from collections import defaultdict
import time
import json 


proc =   c.processes["b3f9c335-7878-4465-b4ba-14dd603f2fbb"]
            
base_name = 'AUT_KM_2022_LAI_10_high_mistral-nemo_simple'
            
print("Task excecution starting...\n")


ts_spec = {

    "process_id": "b3f9c335-7878-4465-b4ba-14dd603f2fbb",
    "name": "TS Imputation LLM Test - test remote",
    "image": "latest",
    "tool":"ts-imputation",
    "inputs": {
        "missing": [
            'bf3e9ef8-85ad-431f-bb8c-3a5684df4e37'
        ],
        "ground_truth": [
            'edf678ed-68fa-4858-a39e-56f6ade77bff'
        ]
    },
    "datasets": {
        "d1": {
            "name":"ts-impute-out",
            "owner_org":"athena-research-center",
            "notes": "Experimental results from TS Imputation",
            "tags":["ARC","TS Imputation"]
        }
    },

    "parameters": {
        "imp_method": "llm_based", #
        "LLM": "zero_shot_prompting", #
        "time":"time", #
        "starting_date": "None", #
        "end_date": "None", #
        "crop_type" : "maize",
        "prompt_mode" : 'simple' ,#
        "inference_engine": "groq", 
        "model_name" : "llama-3.1-8b-instant", 
        "num_of_examples": 1,#
        "time_format": "%Y-%m-%d",
        "sep" : ",",#
        "header" : 0,#
    },

    "secrets": {
        "groq_api_key": "***",  # 
        "api_token": "***",#

    },

    "outputs": {
        "imputed_timeseries": {
            "url": f"s3://klms-bucket/LAI_ts_imputation_imputed_dfs/{base_name}_imputed.csv",
            "dataset": "d1",
            "resource": {
                "name": "Imputed Timeseries",
                "relation": "owned"
            }
        }
    }
}


# Send a Request to the STELAR API to invoke a Task for NPK using the above spec. The task id is kept
ts_spec['process_id'] = str(proc.id)
ts_spec_res = c.POST("v2/task", **ts_spec).json()["result"]

t = c.tasks[ts_spec_res['id']]

t.proxy_sync()

ts_id = str(t.id)
ts_signature = t.signature



while t.exec_state not in ["succeeded", "failed"]:
    t.proxy_sync()
    if t.exec_state == "failed":
        print(t.exec_state)    
    elif t.exec_state == "succeeded":
        print("Task completed successfully.")
    time.sleep(20)



Task excecution starting...

Task completed successfully.

